In [1]:
import pandas as pd
import numpy as np

In [2]:
dataframe = pd.read_csv("flight_data.zip")

In [3]:
dataframe.describe()

,Date of Booking,Date of Journey,Airline-Class,Departure Time,Arrival Time,Duration,Total Stops,Price
count,452088,452088,452088,452088,452088,452088,452088,452088
unique,1,50,1836,1011,1050,482,13,20781
top,15/01/2023,31/01/2023,Vistara \nUK-918\nECONOMY,20:30\nHyderabad,19:50\nChennai,02h 15m,1-stop,"54,879"
freq,452088,9824,2270,5772,5069,4989,366989,2702


In [4]:
dataframe.isnull().sum()

Date of Booking    0
Date of Journey    0
Airline-Class      0
Departure Time     0
Arrival Time       0
Duration           0
Total Stops        0
Price              0
dtype: int64

# Pre-Processing Data

In [5]:
dataframe[['Airline', 'Flight_Code', 'Class']] = dataframe['Airline-Class'].str.split('\n', n=2, expand=True)
dataframe[['Departure_Time', 'Departure_Location']] = dataframe['Departure Time'].str.split('\n', n=1, expand=True)
dataframe[['Arrival_Time', 'Arrival_Location']] = dataframe['Arrival Time'].str.split('\n', n=1, expand=True)

In [6]:
dataframe['Date of Booking'] = pd.to_datetime(dataframe['Date of Booking'],format='%d/%m/%Y')
dataframe['Date of Journey'] = pd.to_datetime(dataframe['Date of Journey'],format='%d/%m/%Y')
dataframe['Booking_Advance'] = (dataframe['Date of Journey'] - dataframe['Date of Booking']).dt.days
dataframe['Journey_day'] = (dataframe['Date of Journey']).dt.dayofweek
dataframe['booking_month'] = dataframe['Date of Booking'].dt.month

In [7]:
dataframe.head(2)

,Date of Booking,Date of Journey,Airline-Class,Departure Time,Arrival Time,Duration,Total Stops,Price,Airline,Flight_Code,Class,Departure_Time,Departure_Location,Arrival_Time,Arrival_Location,Booking_Advance,Journey_day,booking_month
0,2023-01-15,2023-01-16,SpiceJet \nSG-8169\nECONOMY,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,"5,335",SpiceJet,SG-8169,ECONOMY,20:00,Delhi,22:05,Mumbai,1,0,1
1,2023-01-15,2023-01-16,Indigo \n6E-2519\nECONOMY,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,"5,899",Indigo,6E-2519,ECONOMY,23:00,Delhi,01:20,Mumbai,1,0,1


In [8]:
dataframe = dataframe.drop(columns=['Airline-Class','Departure Time','Arrival Time','Date of Booking','Date of Journey','Flight_Code'])

In [9]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Duration            452088 non-null  object
 1   Total Stops         452088 non-null  object
 2   Price               452088 non-null  object
 3   Airline             452088 non-null  object
 4   Class               452088 non-null  object
 5   Departure_Time      452088 non-null  object
 6   Departure_Location  452088 non-null  object
 7   Arrival_Time        452088 non-null  object
 8   Arrival_Location    452088 non-null  object
 9   Booking_Advance     452088 non-null  int64 
 10  Journey_day         452088 non-null  int32 
 11  booking_month       452088 non-null  int32 
dtypes: int32(2), int64(1), object(9)
memory usage: 37.9+ MB


In [10]:
def time_to_minutes(time_str):
    h, m = map(int, time_str.split(':'))
    return round(h + m/60.0, 4)

In [11]:
dataframe['Departure_Time'] = dataframe['Departure_Time'].apply(time_to_minutes)
dataframe['Arrival_Time'] = dataframe['Arrival_Time'].apply(time_to_minutes)

In [12]:
dataframe['Duration'] = dataframe['Duration'].str.extract(r'(\d+)h (\d+)m') \
                                     .astype(float) \
                                     .apply(lambda x: round(x[0] + (x[1]) / 60, 4), axis=1)

In [13]:
dataframe['Price'] = dataframe['Price'].replace('[,]', '', regex=True).astype(int)

In [14]:
dataframe['Total Stops'] = dataframe['Total Stops'].str.replace(r'\n\s*\t*', '', regex=True)
dataframe['Total Stops'] = dataframe['Total Stops'].str.replace(r'(stop).*', r'\1', regex=True)

In [15]:
dataframe['Airline'] = dataframe['Airline'].str.replace(r'\n\s*\t*', '', regex=True)
dataframe['Airline'] = dataframe['Airline'].str.replace(r'(stop).*', r'\1', regex=True)
dataframe['Airline'] = dataframe['Airline'].str.strip()

In [16]:
dataframe.head()

,Duration,Total Stops,Price,Airline,Class,Departure_Time,Departure_Location,Arrival_Time,Arrival_Location,Booking_Advance,Journey_day,booking_month
0,2.0833,non-stop,5335,SpiceJet,ECONOMY,20.0000,Delhi,22.0833,Mumbai,1,0,1
1,2.3333,non-stop,5899,Indigo,ECONOMY,23.0000,Delhi,1.3333,Mumbai,1,0,1
2,2.1667,non-stop,5801,GO FIRST,ECONOMY,22.5000,Delhi,0.6667,Mumbai,1,0,1
3,2.0833,non-stop,5794,SpiceJet,ECONOMY,18.8333,Delhi,20.9167,Mumbai,1,0,1
4,2.1667,non-stop,5955,Air India,ECONOMY,20.0000,Delhi,22.1667,Mumbai,1,0,1


In [17]:
dataframe["Airline"].unique()

array(['SpiceJet', 'Indigo', 'GO FIRST', 'Air India', 'AirAsia',
       'Vistara', 'AkasaAir', 'AllianceAir', 'StarAir'], dtype=object)

In [18]:
vistara_df = dataframe[(dataframe['Airline'] == 'Vistara') & (dataframe['Class'] == 'ECONOMY')]

In [19]:
vistara_df.head(4)

,Duration,Total Stops,Price,Airline,Class,Departure_Time,Departure_Location,Arrival_Time,Arrival_Location,Booking_Advance,Journey_day,booking_month
31,2.2500,non-stop,7320,Vistara,ECONOMY,19.7500,Delhi,22.0000,Mumbai,1,0,1
51,24.7500,1-stop,8055,Vistara,ECONOMY,13.3333,Delhi,14.0833,Mumbai,1,0,1
58,26.5000,1-stop,8370,Vistara,ECONOMY,13.3333,Delhi,15.8333,Mumbai,1,0,1
62,16.0833,1-stop,8491,Vistara,ECONOMY,5.5833,Delhi,21.6667,Mumbai,1,0,1


In [20]:
vistara_df.shape

(83847, 12)

In [21]:
features = vistara_df.drop('Price', axis = 1)
target = vistara_df['Price']

In [22]:
categorical_features = ['Class', 'Total Stops','Airline','Arrival_Location','Departure_Location']
numeric_features = ['Booking_Advance','Arrival_Time','Departure_Time','Duration', 'Journey_day', 'booking_month']

In [23]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [24]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# XGBoost Regression

In [25]:
# Create a pipeline with preprocessing and XGBRegressor model
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [26]:
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

In [27]:
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 4, 5],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.7, 0.8, 0.9],
    'regressor__colsample_bytree': [0.7, 0.8, 0.9]
}

In [28]:
from sklearn.model_selection import GridSearchCV

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, 
                           param_grid=param_grid, 
                           cv=3,  # 3-fold cross-validation
                           scoring='neg_mean_squared_error',  # Metric to optimize
                           n_jobs=-1) 

In [29]:
# Train the model
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Booking_Advance',
                                                                          'Arrival_Time',
                                                                          'Departure_Time',
                                                                          'Duration',
                                                                          'Journey_day',
                                                                          'booking_month']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['Class',
                                                                          'Total '
                                                                          'Stops',
                                                                          'Airline',
                                                                          'Arrival_Location',
                                                                          'Departure_Location'])])),
                                       ('regressor',
                                        XGBRegressor(base_score=N...
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'regressor__colsample_bytree': [0.7, 0.8, 0.9],
                         'regressor__learning_rate': [0.01, 0.1, 0.2],
                         'regressor__max_depth': [3, 4, 5],
                         'regressor__n_estimators': [100, 200, 300],
                         'regressor__subsample': [0.7, 0.8, 0.9]},
             scoring='neg_mean_squared_error')

In [30]:
print(f"Best Parameters: {grid_search.best_params_}")

Best Parameters: {'regressor__colsample_bytree': 0.9, 'regressor__learning_rate': 0.2, 'regressor__max_depth': 5, 'regressor__n_estimators': 300, 'regressor__subsample': 0.8}
